# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图显示了一条位于广阔绿色草地中的木制栈道。栈道直线延伸，引导视线穿过丰富的绿色草丛，直至远处的树木和开阔的蓝天。图像中的天空被淡淡的云朵点缀，呈现出宁静和清新的自然景观。整体上，这张图片给人一种平静和放松的感觉，是自然美景的典型展示。此外，这样的环境可能是野生动植物的栖息地，适合进行自然观察和徒步旅行。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图显示了一条位于广阔绿色草地中的木制栈道。栈道直线延伸，引导视线穿过丰富的绿色草丛，直至远处的树木和开阔的蓝天。图像中的天空被淡淡的云朵点缀，呈现出宁静和清新的自然景观。整体上，这张图片给人一种平静和放松的感觉，是自然美景的典型展示。此外，这样的环境可能是野生动植物的栖息地，适合进行自然观察和徒步旅行。'

### 封装成一个函数 query_image_description

In [1]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这张图片是一种比较幽默的对比图，使用了网络流行的狗头表情包来表现人生不同阶段的状态。图片分为左右两部分，每部分都有文字描述。

左边部分标题是“16岁的我”，图片中的狗头表情包是一只肌肉发达的狗，下面的文字说明了年轻时的状态和心态：

- 我前途一片光明
- 身体素质高
- 八九点钟的太阳
- 未来可期

右边部分标题是“工作后的我”，图片中的狗头表情包是一只看起来有些疲惫、憔悴的狗，下面的文字说明了工作后的状态和心态：

- 好累好困
- 好想睡懒觉
- 重大伤病不要靠近我啊啊啊
- 我好弱小啊啊啊啊啊
- 我就是普通人

这幅图通过对比年轻时的雄心壮志、充满活力和工作后的疲倦、沮丧，反映了许多人在不同人生阶段的心理和生理状态的变化，具有一定的讽刺意味和现实感。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [2]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [3]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图显示了1980年至2020年间美国、中国、日本和德国的GDP（国内生产总值）的比较。以下是对图中各国GDP变化趋势的解释：

1. **美国（蓝线）**：
   - 从1980年到2020年，美国的GDP一直在稳步增长。
   - 1980年，美国的GDP大约是2.8万亿美元。
   - 到2020年，美国的GDP增长到超过20万亿美元，呈现出明显的上升趋势。

2. **中国（红线）**：
   - 从1980年到2000年，中国的GDP增长缓慢。
   - 从2000年左右开始，中国的GDP增速显著加快，尤其是2005年之后增长更为明显。
   - 到2020年，中国的GDP接近15万亿美元，显示出强劲的经济增长。

3. **日本（紫线）**：
   - 从1980年到1990年，日本的GDP快速增长。
   - 1990年之后，日本的GDP增长趋于平缓，并在1995年左右达到峰值。
   - 1995年后，日本的GDP出现了波动，整体上维持在一个较高但平稳的水平。
   
4. **德国（绿线）**：
   - 从1980年到2000年，德国的GDP呈稳定增长趋势。
   - 2000年后，德国的GDP继续增长，但速度较为平稳。
   - 到2020年，德国的GDP接近4万亿美元。

总结：
- **美国**保持了世界最大经济体的位置，其GDP稳步增长。
- **中国**在2000年后经济快速发展，GDP大幅上升，接近美国。
- **日本**在1990年代达到经济高峰后，GDP增长趋缓。
- **德国**的经济表现稳定，GDP增长较为平稳。

这些数据反映了各国在全球经济中的地位和变化趋势。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [ ]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

![](./images/handwriting_1.jpg)

In [5]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示了关于PEFT（Parameter-Efficient Fine-Tuning）方法和多模态指令微调大语言模型（LLMs）的手写笔记。以下是对内容的解释：

### 左侧页面
1. **H-F Transformers**
   - 模型
   - 数据
   - Benchmark

2. **PEFT: SOTA（State Of The Art）**
   - PEFT Methods

3. **PEFT**
   - **Prompt Tuning**
     - Adapter (2019, Google)
     - Prefix (2021, Stanford)
     - Prompt (2021, Google)
     - P-Tuning v1 (2021)
     - P-Tuning v2 (2021)
     - Soft Prompts (2021)
     - Hard Prompts
       - Template-based
   - Instruction Tuning: Ft LLMs
     - T5 (21.09, Google)
     - OpenAI Instruction GPT
     - XRL HF
     - BLOOMZ
     - Alpaca
     - Claude
     - ChatGLM
     - MOSS

### 右侧页面
1. **Multi-modality Instruction Ft. LLMs**
   - LLaVA (13B)

2. **LORA**
   - LORA
   - QLORA
   - AdaLORA

3. **Prefix-tuning & Adapters**
   - Refine Δh in LLMs
     - Δh: Project down -> non-linear -> Project up
   - Insertion form: added type
     - Modified representation: added structure in param
     - Composition func: Δh + h = ?
     - Adapter:
       - Prefix: Δh
       - Scaled parallel adapter
       - MAM Adapters: scaled parallel adapter -> FFN -> MIM -> head attention
       - Soft prompt -> Better results

### 笔记的核心思想
这些笔记讨论了各种参数高效微调（PEFT）方法，特别是提示调优（Prompt Tuning）和适配器（Adapters），以及在大语言模型上的指令微调（Instruction Tuning）。此外，还提到了多模态指令微调（如LLaVA）和一些新的方法如LORA和它的变种（QLORA、AdaLORA）。

笔记中展示了对不同调优技术的简要介绍及其发展时间，特别关注了这些方法在模型参数调整中的角色以及它们的优点。

In [4]:
from IPython.display import display, Markdown

content = query_base64_image_description("./images/gpt-4v.png")
display(Markdown(content))

这张图显示的是一些手写的文字和箭头。内容似乎与某种过程或计算有关，可能是物理或工程领域的。其中一些文字是“max”、“Q”和“Vmax”，这些术语可能与最大值或最大量相关。文字的排列显示了一个从左向右的顺序或转换过程，各部分之间用箭头连接表示转换或流程的方向。

具体来说，“Vmax”可能指的是“最大速度”或“最大电压”，具体含义取决于上下文。而“Q”可能代表“量”或“电荷”，同样也需要更多上下文来准确解释。总体上，这张图片看起来像是快速草绘的笔记或计算过程。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。